## ДЗ_8

### Выяснить какая архитектура больше подходит для задачи сентимент анализа на данных - CNN или RNN

*1. Построить свёрточную архитектуру*

*2. Построить различные архитектуры с RNN*

*3. Построить совместные архитектуры CNN -> RNN и/или (RNN -> CNN)*

*4. Сделать выводы что получилось лучше*

**Загрузим необходимые библиотеки и скачаем данные.**

In [6]:
!pip install stop_words
!pip install pymorphy2
!pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.3/612.3 kB 8.3 MB/s eta 0:00:00


In [7]:
import pandas as pd
import numpy as np

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

import keras
import tensorflow as tf
import tensorflow_addons as tfa
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
tqdm.pandas()

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [8]:
df_train = pd.read_csv("./train.csv")
df_test = pd.read_csv("./test.csv")
df_val = pd.read_csv("./val.csv")

In [9]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [10]:
df_test.head()

,id,text
0,204150,Тектоника и рельеф-самое ужасное в мире мучение(
1,204151,"Ходили запускать шар желаний, но у нас не полу..."
2,204152,"Хочу лето только ради того, что бы направить н..."
3,204153,RT @RonyLiss: @colf_ne блин((\nа я шипперила Ф...
4,204154,"RT @anna_romt: @ZADROT_PO_IGRAM блин,каждое во..."


In [11]:
df_val.head()

,id,text,class
0,181467,RT @TukvaSociopat: Максимальный репост! ))) #є...,1
1,181468,чтоб у меня з.п. ежегодно индексировали на инд...,0
2,181469,@chilyandlime нехуя мне не хорошо !!! :((((,0
3,181470,"@inafish нее , когда ногами ахахах когда?ахаха...",0
4,181471,"Хочу сделать как лучше, а получаю как всегда. :(",0


In [12]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].progress_apply(preprocess_text)
df_val['text'] = df_val['text'].progress_apply(preprocess_text)
df_test['text'] = df_test['text'].progress_apply(preprocess_text)

100%|██████████| 22684/22684 [00:31<00:00, 722.41it/s]


In [13]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [14]:
tokenizer = Tokenizer(num_words=None,
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [15]:
len(tokenizer.index_word)

258107

In [16]:
sequences_train[:10]

[[66107, 66108, 4712, 582, 3, 596],
 [1, 28151, 214, 6915, 451, 173, 9, 39004],
 [1, 1467, 1875, 654, 571, 1],
 [1, 66109, 41, 5700, 39005, 28152, 11483],
 [66110, 18433, 66111, 66112, 148, 362],
 [12632, 36, 123, 5, 24],
 [1, 66113, 15886, 7619, 22180, 67, 2150, 95, 3481, 27, 2150, 12633, 856],
 [169, 368, 3002, 66114, 3644, 113, 22181, 66115],
 [66116, 8571, 1185, 89],
 [12634, 4713, 2689, 39006]]

In [17]:
word_count, training_length

(258108, 27)

In [18]:
y_train = df_train['class'].values
y_val = df_val['class'].values

**Чтобы определить какая их архитектур нейронных сетей лучше справляется с поставленной задачей, построим несложную CNN-сеть и на ее основе проведем эксперименты - будем добавлять слои с другими сетями, при этом оставляя параметры сетей максимально похожими. Чтобы происходило соревнование алгоритмов.**

### CNN

In [19]:
model = Sequential()
model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(Conv1D(64, 3))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 27, 30)            7743240   
                                                                 
 masking (Masking)           (None, 27, 30)            0         
                                                                 
 conv1d (Conv1D)             (None, 25, 128)           11648     
                                                                 
 dropout (Dropout)           (None, 25, 128)           0         
                                                                 
 activation (Activation)     (None, 25, 128)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 23, 64)            24640     
                                                                 
 dropout_1 (Dropout)         (None, 23, 64)            0

In [20]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=1)])

In [21]:
early_stopping=EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=5,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/5
319/319 [==============================] - 67s 203ms/step - loss: 0.6921 - accuracy: 0.5117 - f1_score: 0.6734 - val_loss: 0.6918 - val_accuracy: 0.5165 - val_f1_score: 0.6710
Epoch 2/5
319/319 [==============================] - 65s 204ms/step - loss: 0.6909 - accuracy: 0.5319 - f1_score: 0.6734 - val_loss: 0.6907 - val_accuracy: 0.5423 - val_f1_score: 0.6710
Epoch 3/5
319/319 [==============================] - 64s 200ms/step - loss: 0.6892 - accuracy: 0.5566 - f1_score: 0.6734 - val_loss: 0.6895 - val_accuracy: 0.5427 - val_f1_score: 0.6710
Epoch 4/5
319/319 [==============================] - 65s 203ms/step - loss: 0.6871 - accuracy: 0.5650 - f1_score: 0.6734 - val_loss: 0.6879 - val_accuracy: 0.5524 - val_f1_score: 0.6710
Epoch 5/5
319/319 [==============================] - 65s 203ms/step - loss: 0.6844 - accuracy: 0.5763 - f1_score: 0.6734 - val_loss: 0.6855 - val_accuracy: 0.5609 - val_f1_score: 0.6710


In [22]:
score_cnn = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score_cnn[0])
print('Test accuracy:', score_cnn[1])

45/45 [==============================] - 1s 29ms/step - loss: 0.6879 - accuracy: 0.5451 - f1_score: 0.6709


Test score: 0.6878589987754822
Test accuracy: 0.5451219081878662


### RNN

In [23]:
model = Sequential()
model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(128))
model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 27, 30)            7743240   
                                                                 
 masking_1 (Masking)         (None, 27, 30)            0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               20352     
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 activation_4 (Activation)   (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 64)               

In [24]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=1)])

In [25]:
early_stopping=EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=5,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/5
319/319 [==============================] - 63s 185ms/step - loss: 0.6923 - accuracy: 0.5179 - f1_score: 0.6734 - val_loss: 0.6881 - val_accuracy: 0.5627 - val_f1_score: 0.6710
Epoch 2/5
319/319 [==============================] - 59s 187ms/step - loss: 0.6874 - accuracy: 0.5528 - f1_score: 0.6734 - val_loss: 0.6831 - val_accuracy: 0.5741 - val_f1_score: 0.6710
Epoch 3/5
319/319 [==============================] - 60s 187ms/step - loss: 0.6827 - accuracy: 0.5667 - f1_score: 0.6734 - val_loss: 0.6792 - val_accuracy: 0.5741 - val_f1_score: 0.6710
Epoch 4/5
319/319 [==============================] - 58s 182ms/step - loss: 0.6790 - accuracy: 0.5737 - f1_score: 0.6734 - val_loss: 0.6763 - val_accuracy: 0.5786 - val_f1_score: 0.6710
Epoch 5/5
319/319 [==============================] - 59s 184ms/step - loss: 0.6752 - accuracy: 0.5796 - f1_score: 0.6734 - val_loss: 0.6725 - val_accuracy: 0.5826 - val_f1_score: 0.6710


In [26]:
score_rnn = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score_rnn[0])
print('Test accuracy:', score_rnn[1])

45/45 [==============================] - 1s 30ms/step - loss: 0.6892 - accuracy: 0.5486 - f1_score: 0.6709


Test score: 0.6891513466835022
Test accuracy: 0.5486046671867371


### GRU

In [27]:
model = Sequential()
model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(GRU(128,recurrent_dropout=0.2))
model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 27, 30)            7743240   
                                                                 
 masking_2 (Masking)         (None, 27, 30)            0         
                                                                 
 gru (GRU)                   (None, 128)               61440     
                                                                 
 dense_6 (Dense)             (None, 128)               16512     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 activation_8 (Activation)   (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 64)               

In [28]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=1)])

In [29]:
early_stopping=EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=5,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/5
319/319 [==============================] - 132s 392ms/step - loss: 0.6928 - accuracy: 0.5177 - f1_score: 0.6734 - val_loss: 0.6924 - val_accuracy: 0.5268 - val_f1_score: 0.6710
Epoch 2/5
319/319 [==============================] - 123s 385ms/step - loss: 0.6914 - accuracy: 0.5445 - f1_score: 0.6734 - val_loss: 0.6900 - val_accuracy: 0.5647 - val_f1_score: 0.6710
Epoch 3/5
319/319 [==============================] - 126s 394ms/step - loss: 0.6866 - accuracy: 0.5876 - f1_score: 0.6734 - val_loss: 0.6821 - val_accuracy: 0.6016 - val_f1_score: 0.6710
Epoch 4/5
319/319 [==============================] - 123s 385ms/step - loss: 0.6738 - accuracy: 0.6141 - f1_score: 0.6734 - val_loss: 0.6655 - val_accuracy: 0.6171 - val_f1_score: 0.6710
Epoch 5/5
319/319 [==============================] - 125s 393ms/step - loss: 0.6511 - accuracy: 0.6361 - f1_score: 0.6734 - val_loss: 0.6413 - val_accuracy: 0.6382 - val_f1_score: 0.6710


In [30]:
score_gru = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score_gru[0])
print('Test accuracy:', score_gru[1])

45/45 [==============================] - 3s 74ms/step - loss: 0.6557 - accuracy: 0.6086 - f1_score: 0.6709


Test score: 0.655731737613678
Test accuracy: 0.6086496710777283


### LSTM

In [31]:
model = Sequential()
model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(LSTM(128, recurrent_dropout=0.2))
model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 27, 30)            7743240   
                                                                 
 masking_3 (Masking)         (None, 27, 30)            0         
                                                                 
 lstm (LSTM)                 (None, 128)               81408     
                                                                 
 dense_10 (Dense)            (None, 128)               16512     
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 activation_12 (Activation)  (None, 128)               0         
                                                                 
 dense_11 (Dense)            (None, 64)               

In [32]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=1)])

In [33]:
early_stopping=EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=5,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/5
319/319 [==============================] - 153s 468ms/step - loss: 0.6928 - accuracy: 0.5342 - f1_score: 0.6734 - val_loss: 0.6922 - val_accuracy: 0.5644 - val_f1_score: 0.6710
Epoch 2/5
319/319 [==============================] - 154s 482ms/step - loss: 0.6910 - accuracy: 0.5735 - f1_score: 0.6734 - val_loss: 0.6891 - val_accuracy: 0.5802 - val_f1_score: 0.6710
Epoch 3/5
319/319 [==============================] - 149s 468ms/step - loss: 0.6857 - accuracy: 0.5878 - f1_score: 0.6734 - val_loss: 0.6806 - val_accuracy: 0.5913 - val_f1_score: 0.6710
Epoch 4/5
319/319 [==============================] - 153s 478ms/step - loss: 0.6734 - accuracy: 0.5970 - f1_score: 0.6734 - val_loss: 0.6670 - val_accuracy: 0.6056 - val_f1_score: 0.6710
Epoch 5/5
319/319 [==============================] - 151s 473ms/step - loss: 0.6584 - accuracy: 0.6125 - f1_score: 0.6734 - val_loss: 0.6514 - val_accuracy: 0.6201 - val_f1_score: 0.6710


In [34]:
score_lstm = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score_lstm[0])
print('Test accuracy:', score_lstm[1])

45/45 [==============================] - 4s 85ms/step - loss: 0.6704 - accuracy: 0.5906 - f1_score: 0.6709


Test score: 0.6704389452934265
Test accuracy: 0.5905744433403015


### CNN > RNN

In [35]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(Conv1D(128, 3))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(LSTM(64,recurrent_dropout=0.2))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 27, 30)            7743240   
                                                                 
 masking_4 (Masking)         (None, 27, 30)            0         
                                                                 
 conv1d_2 (Conv1D)           (None, 25, 128)           11648     
                                                                 
 dropout_8 (Dropout)         (None, 25, 128)           0         
                                                                 
 activation_16 (Activation)  (None, 25, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense_14 (Dense)            (None, 64)               

In [36]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=1)])

In [37]:
early_stopping=EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=5,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/5
319/319 [==============================] - 120s 363ms/step - loss: 0.6926 - accuracy: 0.5508 - f1_score: 0.6734 - val_loss: 0.6919 - val_accuracy: 0.5610 - val_f1_score: 0.6710
Epoch 2/5
319/319 [==============================] - 118s 369ms/step - loss: 0.6907 - accuracy: 0.5659 - f1_score: 0.6734 - val_loss: 0.6889 - val_accuracy: 0.5632 - val_f1_score: 0.6710
Epoch 3/5
319/319 [==============================] - 116s 362ms/step - loss: 0.6865 - accuracy: 0.5745 - f1_score: 0.6734 - val_loss: 0.6834 - val_accuracy: 0.5762 - val_f1_score: 0.6710
Epoch 4/5
319/319 [==============================] - 115s 362ms/step - loss: 0.6790 - accuracy: 0.5832 - f1_score: 0.6734 - val_loss: 0.6744 - val_accuracy: 0.5897 - val_f1_score: 0.6710
Epoch 5/5
319/319 [==============================] - 117s 366ms/step - loss: 0.6681 - accuracy: 0.5975 - f1_score: 0.6734 - val_loss: 0.6638 - val_accuracy: 0.6007 - val_f1_score: 0.6710


In [38]:
score_cnn_rnn = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score_cnn_rnn[0])
print('Test accuracy:', score_cnn_rnn[1])

45/45 [==============================] - 4s 76ms/step - loss: 0.6825 - accuracy: 0.5671 - f1_score: 0.6709


Test score: 0.6824623942375183
Test accuracy: 0.5671207308769226


### RNN > CNN

In [39]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(128,recurrent_dropout=0.2, return_sequences=True))
model.add(Conv1D(128, 3))
model.add(Dropout(0.2))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 27, 30)            7743240   
                                                                 
 masking_5 (Masking)         (None, 27, 30)            0         
                                                                 
 lstm_2 (LSTM)               (None, 27, 128)           81408     
                                                                 
 conv1d_3 (Conv1D)           (None, 25, 128)           49280     
                                                                 
 dropout_10 (Dropout)        (None, 25, 128)           0         
                                                                 
 global_max_pooling1d_1 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                      

In [40]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=1)])

In [41]:
early_stopping=EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=5,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/5
319/319 [==============================] - 191s 586ms/step - loss: 0.6927 - accuracy: 0.5136 - f1_score: 0.6734 - val_loss: 0.6925 - val_accuracy: 0.5049 - val_f1_score: 0.6710
Epoch 2/5
319/319 [==============================] - 182s 569ms/step - loss: 0.6914 - accuracy: 0.5365 - f1_score: 0.6734 - val_loss: 0.6910 - val_accuracy: 0.5141 - val_f1_score: 0.6710
Epoch 3/5
319/319 [==============================] - 183s 575ms/step - loss: 0.6878 - accuracy: 0.5746 - f1_score: 0.6734 - val_loss: 0.6860 - val_accuracy: 0.5824 - val_f1_score: 0.6710
Epoch 4/5
319/319 [==============================] - 183s 575ms/step - loss: 0.6772 - accuracy: 0.6002 - f1_score: 0.6734 - val_loss: 0.6736 - val_accuracy: 0.6036 - val_f1_score: 0.6710
Epoch 5/5
319/319 [==============================] - 182s 572ms/step - loss: 0.6622 - accuracy: 0.6080 - f1_score: 0.6734 - val_loss: 0.6623 - val_accuracy: 0.6176 - val_f1_score: 0.6710


In [42]:
score_rnn_cnn = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score_rnn_cnn[0])
print('Test accuracy:', score_rnn_cnn[1])

45/45 [==============================] - 5s 109ms/step - loss: 0.6724 - accuracy: 0.5842 - f1_score: 0.6709


Test score: 0.6724264621734619
Test accuracy: 0.5842260718345642


**Для удобства выведем результаты в таблицу:**

In [43]:
result = pd.DataFrame({'model': ['CNN', 'RNN', 'GRU', 'LSTM','CNN_RNN', 'RNN_CNN'],
                       'accuracy': [score_cnn[1], score_rnn[1], score_gru[1],
                                    score_lstm[1], score_cnn_rnn[1], score_rnn_cnn[1]]})
result.sort_values(by='accuracy', ascending=False)

,model,accuracy
2,GRU,0.608650
3,LSTM,0.590574
5,RNN_CNN,0.584226
4,CNN_RNN,0.567121
1,RNN,0.548605
0,CNN,0.545122


#### ВЫВОД: результаты экспермента показывают, что при максимально равных условиях лучше всего справляются с поставленной задачей нейронные сети, способные бороться с "затуханием/взрывом" градиента (LSTM, GRU). Обе они лидируют с заметным отрывом и имеют практически одинаковый показатель "accuracy". На втором месте - совместные архитектуры (RNN_CNN, CNN_RNN). Они так же имеют очень похожий друг с другом показатель "accuracy", что подталкивает к выводу - не важно какой слой следует за каким. Результат получается одинаковым. И, неожиданно, на третьем месте оказались сети CNN и RNN. Их "accuracy" тоже практически одинаковый. В результате простая сеть с простой архитектурой показала чуть лучший результат.